# Preparing Training Data (remote sensing) for DeepLab

Training data is critical for supervised learning. There are many public training data available online, such as [ImageNet](http://www.image-net.org/) and [PASCAL VOC 2012](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/), can be used to train deep learning algorithms. Images in these datasets are from everyday scenes and contain targets such as cats, cars, and people.   

We need to prepare our new, unique training data because both images and targets are different from those in the public datasets. We use images from satellites or airborne platforms, and targets are landforms, landslides, and other objects on the surface. 

To understand how we prepare trianing data for DeepLab, we will go through each step in this tutorial. [DeepLab](https://github.com/tensorflow/models/tree/master/research/deeplab) is a supervised learning algorithm for labeling targets on images. An UAV-derived image covering Eboling Mountain on the Tibetan Plateau will be used for the demonstration. Please refer to [Huang et al., 2018](https://www.mdpi.com/2072-4292/10/12/2067) for more information.

## A glimpse on training data

#### Training images and the corresponding label images from PASCAL VOC 2012:
![title](figures/voc_air_plane_layout_trim.png ) 
In this example, the label image (pixel-wise class map) shows where are the targets (airple) and background (sky). Usually, the pixels in the label images store the class ID (an integer), which is assigned with a specific color (e.g., red) for visualization. These label images were created by humans using annotation tools such as [labelme](https://github.com/wkentaro/labelme).

#### Training polygons and remote sensing images:

 <img src=figures/training_polygon_source_trim.jpg width="800"/>
The figure above shows the distribution of the training polygons: 15 of them are non-gullies; seven were collected from field GPS measurements; and four were from manual delineation (digitizing).
 
Compared with images from everyday scenes, remote sensing images have characteristics of a large size (e.g., > 10000 by 10000 pixels), multiple bands (>= three), and coordinates. The coordinates are very important because they represent target locations on the earth's surface.

Two categories of training polygons: positive and negative polygons, are presented in the figure above. Positive ones are target boundaries (e.g., gully polygons), and negative ones are these covering non-target areas.

<!-- #### Training data derived from remote sensing images:  -->


## Step 1: delineating training polgyons

As mentioned in the previous cell, positive training polygons are target boundaries can be delineated on images or measured in the field. These postitive ones are also ground truhts used from validation of the mapping algorithm. If someone else already prepared these ground truths, then you can take them and save a lot of time. 

In the Eboling study, the ground truth polygons were collected using the Real-time Kinematic (RTK) measurements in the field and manually delineated (digitizing) on the UAV-derived images(aka DOM). 

The steps of delineation of polygons in [QGIS](https://www.qgis.org) include: (1) "New Shapefile Layer" and choosing "Type" as "Polygon"; (2) adding a new field name "class_int" to the shapefile; (3) "Add feature", i.e., adding polygons to the shapefile layer and inputitng "class id". 


We also need to add some negative training polygons because they help distinguish the gullies from the similar non-gully land covers during the training. One practical way to add negative ones and make them representative for training is that: running an initial mapping exercise by only using ground truths as training polygons, then adding negative ones to cover areas containing numerous false positives. Otherwise, creating negative training polygons requires expertise and ground knowledge of all the land covers in the study area.

Alternatively, we can prepare positive and negative training polygons separately and save them to different shapefiles, then use [merge_neg_pos_polygons.py](https://github.com/yghlc/ChangeDet_DL/blob/master/dataTools/merge_neg_pos_polygons.py) to merge them. In this case, people don't need to input "class id" because the script will take care of it. 


### A invalid polygon

Sometimes, there could be an error about invalid polygons. For example, an error reported a self-intersection at point '668045.59660145082 4209336.0392965544'

In QGIS, zoom in to this point using a tool named [zoomtocoordinates](https://plugins.qgis.org/plugins/zoomtocoordinates/), then we can see the self-intersection as the figure below:
<img src=figures/self-intersection_example.png width="500" />

Then fix this issue by eidting the polygon and remove the self-intersection. After fixing the error, we run the script again.


## Step 2: setting an area parameter file

We can set the path of remote sensing imagery and training polygons to area parameter files such as [area_Willow_River.ini](https://github.com/yghlc/Landuse_DL/blob/master/ini_files/area_Willow_River.ini)

In the area*.ini file, parameters for training data include:
1. area_name
2. area_time
3. area_remark
4. input_image_dir
5. input_image_or_pattern
6. training_polygons
7. training_polygons_sub

In the area*.ini file, we can also set image paths for inference (prediction)
8. inf_image_dir
9. inf_image_or_pattern

For a centain area, if we have supplimentary data such as DEM for post-processing, we can set their path:
10. dem_file_dir
11. dem_file_or_pattern
12. slope_file_dir
13. slope_file_or_pattern
14. aspect_file_dir
15. aspect_file_or_pattern
16. dem_diff_file_dir
17. dem_diff_file_or_pattern

For a certain area, if there is ground truth available, we can set it here:
18. validation_shape


We can set multiple area parameter files if we have data from multple regions, observations, or times. An exmaple of area parameter files as the following:


In [ ]:
#the parameter file for define a study area (images and training polygons)

##############################################################
###input and output setting

# the name of the study area (no space)
area_name = Willow_River

# the time of the observation for this area (area_remark may also has time information)
area_time = 2020

# additional information about the areas (e.g.  image and acquisition date)
area_remark = rgb

# use the input_image_dir and input_image_pattern for get image list, from which we derive training and label images
input_image_dir = ~/Data/Arctic/canada_arctic/Willow_River/Planet2020
#input_image_or_pattern = *_8bit_rgb.tif
input_image_or_pattern = 20200818_mosaic_8bit_rgb.tif

# training_polygons includes all the training polygons (ground truth and non-ground truth)
training_polygons= ~/Data/Arctic/canada_arctic/Willow_River/training_polygons/WR_training_polygons_v3.shp
# training_polygons_sub for getting the image subset. (equal to or a portion of the all training polygons)
training_polygons_sub = ~/Data/Arctic/canada_arctic/Willow_River/training_polygons/WR_training_polygons_v3.shp

# use the inf_image_dir and inf_image_pattern to get image list for prediction
inf_image_dir = ~/Data/Arctic/canada_arctic/Willow_River/Planet2020
inf_image_or_pattern = *_8bit_rgb.tif

## the following for calculating topography information,
dem_file_dir = ~/Data/Arctic/canada_arctic/DEM/WR_dem_ArcticDEM_mosaic
dem_file_or_pattern = WR_extent_2m_v3.0_ArcticTileDEM_sub_1_prj.tif

slope_file_dir = ~/Data/Arctic/canada_arctic/DEM/WR_dem_ArcticDEM_mosaic
slope_file_or_pattern = WR_extent_2m_v3.0_ArcticTileDEM_sub_1_slope_prj.tif

aspect_file_dir =
aspect_file_or_pattern =

# DEM difference files
dem_diff_file_dir = ~/Data/Arctic/canada_arctic/DEM/WR_dem_diff
dem_diff_file_or_pattern = WR_dem_diff_DEM_diff_nocoreg_prj.tif


# validation files (ground truth) for evaluation
validation_shape = ~/Data/Arctic/canada_arctic/Willow_River/Willow_River_Thaw_Slumps.shp

# define the Cartesian (XY) projection, the input data will be checked and reprojected
cartensian_prj =

## Step 3: generating training data similar to PASCAL VOC 2012 dataset

Because DeepLab only accept training data like PASCAL VOC dataset, we need to convert the training polygons and remote sensing images to this format. 

Make sure that the training data and remote sensing images have the same projection (Coordinate System). The script will check this and raise an error if find inconsistency.

Step 3 will be conducted by the following scripts in [exe.sh](https://github.com/yghlc/Landuse_DL/blob/master/working_dir/exe.sh)

## Data augmentation

To increase to volume and diversity of training data, we perform data agumentation for the image patch, the parameters for data augmentation in "main_para.ini" file include:
1. data_augmentation
2. data_aug_ignore_classes



After subdividing, there are many image and label patches in folder *split_images* and *split_labels*.

In [ ]:
# note: we are not running the following script here, but will run them together with training and prediction scripts. 

# Landuse_DL scripts dir
eo_dir=~/codes/PycharmProjects/Landuse_DL

# remove previous data or results if necessary
#rm time_cost.txt || true
!${eo_dir}/workflow/remove_previous_data.py ${para_file}

#extract sub_images based on the training polgyons
!${eo_dir}/workflow/get_sub_images_multi_regions.py ${para_file}

################################################
## preparing training images.
# there is another script ("build_RS_data.py"), but seem have not finished.

${eo_dir}/workflow/split_sub_images.py ${para_file}
${eo_dir}/workflow/training_img_augment.py ${para_file}
${eo_dir}/workflow/split_train_val.py ${para_file}

## (optional) check sub-images the corresponding label raster

QGIS is the best way for visualiaztion, as follows, 
<img src=figures/qgis_label_raster.png width="600" />

The coordinates, pixel values, and training polygons can be checked together in QGIS.



We also can visualize these patches in QGIS. Notice that, the patches saved by data augmentation script lost their coordinates. It's ok because they are only for the purpose of training, which does not require coordinates. 

## Summary

1. We go through the steps of preparation of training data. 
2. Jupyter notebook is good for demonstration and explanation. However, when I run this script, I would like to put all the script into one bash file, then run the bash file.

